Use https://github.com/rmorshea/traitlets/tree/5.0-prepre-release

+ Add the remote
+ Create a new branch
+ Pull from the remote and the branch 5.0-prepre-release

In [1]:
from traitlets import describe

---

# [Decorators](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)

A decorator is a special syntax which uses the symbol "`@`" to specify that a function should be wrapped by another.

In [2]:
def stringify(function):
    """A decorator that turns outputs into strings"""
    def wrapper(*args, **kwargs):
        print("entered wrapper")
        result = function(*args, **kwargs)
        print("exiting wrapper")
        return str(result)
    return wrapper

In [3]:
@stringify
def my_function(x):
    return x**2

In [4]:
result = my_function(5)
print("-------------------------------")
print("got %s" % describe("the", result))

entered wrapper
exiting wrapper
-------------------------------
got the str '25'


## What if we didn't have decorators?

The code below replicates the result of `@stringify` but without the decorator syntax:

```python
def my_function(x):
    print("called my_function")
    return x**2

my_function = stringify(my_function)
```

---

# [Descriptors](https://docs.python.org/3/howto/descriptor.html)

Which has at least one of three methods,

In [5]:
class Descriptor(object):

    def __set__(self, obj, value):
        print("setting %r to %r" % (value, obj))

    def __get__(self, obj, cls):
        print("getting from %r" % obj)

    def __delete__(self, obj):
        print("deleting from %r" % obj)

and is assigned as a class attribute

In [6]:
class MyClass(object):
    x = Descriptor()

in order to define special logic whenever `getattr`, `setattr`, or `delattr` is called on an owner instance.

In [7]:
mc = MyClass()
mc.x
mc.x = "a value"
del mc.x

getting from <__main__.MyClass object at 0x103514470>
setting 'a value' to <__main__.MyClass object at 0x103514470>
deleting from <__main__.MyClass object at 0x103514470>


---

# [Metaclasses](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/)

Metaclasses are to classes, as classes are to instances.

In python, `type` is the one metaclass to rule them all, because **all** classes in python are instances of it. Even `type` is an instance of `type`!

In [8]:
for cls in (int, str, object, type):
    print("It is %s that %r is an instance of %r" % (isinstance(cls, type), cls, type))

It is True that <class 'int'> is an instance of <class 'type'>
It is True that <class 'str'> is an instance of <class 'type'>
It is True that <class 'object'> is an instance of <class 'type'>
It is True that <class 'type'> is an instance of <class 'type'>


To use a metaclass we define a class which inherits from `type`. This class does not have to have an `__init__` or `__new__` method, but it's not likely that you really needed a metaclass in the first place if you aren't going to use them. After all, it's these methods that allow one to initialize classes.

---

# How does `traitlets` work?

`traitlets` uses metaclasses, descriptors, and decorators in tandem to create special validation and notification phases when setting values.

Stripping away all the extra logic, this is the underlying skeleton of `traitlets`:

In [9]:
import six

class Feature(object):
    
    klass = None
    
    def class_init(self, cls, name):
        print("the feature %r was assigned to %r" % (self, name))
        self.this_class = cls
        self.name = name
    
    def __set__(self, obj, value):
        if isinstance(value, self.klass):
            obj.__dict__[self.name] = value
        else:
            raise TypeError("Expected a %r instance, not %s" % (self.klass, describe("the", value)))
    
    def __get__(self, obj, cls):
        if obj is None:
            # this is true when accessing the attributed
            # from the class, not instances of the class
            return self
        return obj.__dict__[self.name]
    
    def __delete__(self, obj):
        del obj.__dict__[self.name]


class MetaHasFeatures(type):

    def __init__(cls, name, bases, classdict):
        print("creating %r" % cls)
        for k, v in classdict.items():
            if isinstance(v, Feature):
                v.class_init(cls, k)


class HasFeatures(six.with_metaclass(MetaHasFeatures, object)):
    pass

creating <class '__main__.HasFeatures'>


### Now let's use it!

In [10]:
class MyInt(Feature):
    # this feature only accepts ints
    klass = int


class MyClass(HasFeatures):
    
    i = MyInt()

print("-------------------------------")

mc = MyClass()
try:
    mc.i = "1"
except Exception as e:
    print(e)
print("-------------------------------")
mc.i = 1
print("assigned the value %r" % 1)
print("-------------------------------")
print("get %r from %r" % (mc.i, mc))

creating <class '__main__.MyClass'>
the feature <__main__.MyInt object at 0x103524b00> was assigned to 'i'
-------------------------------
Expected a <class 'int'> instance, not the str '1'
-------------------------------
assigned the value 1
-------------------------------
get 1 from <__main__.MyClass object at 0x103524ba8>


---

# What about all the decorators?

<span style="color:red">If the following code doesn't make sense now, don't worry - it's complicated</span>.

In order to make a decorator for observing trait changes we have to make some interelated changes:

1. A `BaseDecorator` object.
    + When called, it must still act like a method.
2. An `observe` decorator.
    + Wraps a method in an `Observer` object.
3. An `Observer` object.
    + After the instance is created it must register itself as an observer.
4. An `OtherFeature` descriptor.
    + It will notify its owner whenever a value changes
5. A new `HasOtherFeatures` class.
    + It will find `BaseObservers` and call their `instance_init` methods so they can register themselves
    + Offer a method `observe` that will register observers
    + Offer a `notify` which will be used to pass change data to observers.

In [11]:
import types
import inspect


class BaseDecorator(object):

    def __init__(self, name, method):
        self.feature_name = name
        self.method = method
    
    def __call__(self, *args, **kwargs):
        return self.method(*args, **kwargs)
    
    def __get__(self, obj, cls):
        if obj is None:
            return self
        return types.MethodType(self.method, obj)


def observe(name):
    def setup(method):
        return Observer(name, method)
    return setup


class Observer(BaseDecorator):

    def instance_init(self, obj):
        obj.observe(self, self.feature_name)


class OtherFeature(Feature):
    
    def __set__(self, obj, new):
        old = obj.__dict__.get(self.name, None)
        super(OtherFeature, self).__set__(obj, new)
        if new != old:
            data = dict(old=old, new=new, name=self.name, owner=obj)
            obj.notify(data)


class HasOtherFeatures(HasFeatures):

    def __init__(self):
        self._observers = {}
        self.setup()
    
    def setup(self):
        for k, v in inspect.getmembers(type(self)):
            if isinstance(v, BaseDecorator):
                v.instance_init(self)
    
    def observe(self, handler, name):
        if name in self._observers:
            olist = self._observers[name]
        else:
            olist = []
            self._observers[name] = olist
        if handler not in olist:
            olist.append(handler)
    
    def notify(self, data):
        for o in self._observers.get(data["name"], []):
            if isinstance(o, BaseDecorator):
                o(self, data)
            else:
                o(data)

creating <class '__main__.HasOtherFeatures'>


### Now let's use it!

In [12]:
class MyInt(OtherFeature):
    # this feature only accepts ints
    klass = int


class MyClass(HasOtherFeatures):
    
    i = MyInt()
    
    @observe("i")
    def _i_observer(self, data):
        print("observed a change!")
        print(data)

print("-------------------------------")

mc = MyClass()
mc.i = 1
print("-------------------------------")

creating <class '__main__.MyClass'>
the feature <__main__.MyInt object at 0x103543860> was assigned to 'i'
-------------------------------
observed a change!
{'old': None, 'name': 'i', 'new': 1, 'owner': <__main__.MyClass object at 0x103524cc0>}
-------------------------------


---

# [The Traitlets Package](https://traitlets.readthedocs.io/en/stable/)

Now that we have the fundamentals down, let's look at traitlets iteslf

In [13]:
from traitlets import *

---

## Defaults

Default value generators create values "on request". In other words, it's only when you call `getattr`
that the method `MyClass._i_default` will be used to create the default. Whatever it returns will be validated, assigned as the current value of the trait, and then returned to the getattr call.

## Validation

Traits have internal validation and cross validation. The interal stages occur first, and are the same for all instances of a trait type. Cross validators are created by users with the `@validate` decorator - we call them this because a validator which is a method on the owner of the trait can check to see whether the value is correct with respect to the state of the instance itself - say to check whether a value falls between the `min` and `max` attributes on the owner instance. Cross validation occurs after internal validation, so a cross validator of an `Int` trait can already expect that the values it recieves are `int` instances. No need to check whether `isinstance(proposal.value, int)`.

## Observers

Events can be observed by specifying trait names, traits with certain metadata, and/or events of a certain type. These events are triggered by passing a bunch to the method `HasTraits.notify_change`. So if you need to create your own kinds of events that people can listen into you need only define a bunch with a 'name' and a 'type'.

## Basic Example:

In [14]:
class MyClass(HasTraits):

    i = Int()

    @default("i")
    def _i_default(self):
        """The default value for the trait 'i'
        """
        print("generating default value")
        return 1
    
    @validate("i")
    def _i_validation(self, proposal):
        """A "cross validator" for the trait 'i'
        """
        print("cross validating %r" % proposal.value)
        return proposal.value
    
    @observe("i", type="change")
    def _i_observer(self, change):
        """An observer for the trait 'i'
        """
        print("observed a change from %r to %r" % (change.old, change.new))

In [15]:
mc = MyClass()

print("got the default value %r" % mc.i)
print("-------------------------------")
try:
    mc.i = "1"
except TraitError:
    print("get an error assigning strings")
print("-------------------------------")
mc.i = 1
print("assigned, but no change occured")
print("-------------------------------")
mc.i = 2
print("-------------------------------")

generating default value
cross validating 1
got the default value 1
-------------------------------
get an error assigning strings
-------------------------------
cross validating 1
assigned, but no change occured
-------------------------------
cross validating 2
observed a change from 1 to 2
-------------------------------


### Register observers on the fly

In [16]:
def on_the_fly_observer(change):
    print("caught the same change on the fly" % change)

mc.observe(on_the_fly_observer, "i")
mc.i = 3

cross validating 3
observed a change from 2 to 3
caught the same change on the fly


---

# Work In Progress... Ignore All Below This Point...

In [17]:
import functools
from traitlets import *


def go_between(public, proxy=None):
    return GoBetween(public, proxy or "")


class GoBetween(DefaultHandler):

    def __init__(self, public, proxy):
        super(GoBetween, self).__init__([public], tags=None)
        self.trait_name = self.trait_names[0]
        self.proxy = proxy

    def _init_call(self, func):
        @functools.wraps(func)
        def wrapper(owner):
            if not hasattr(owner, self.proxy):
                # Solely in order to assign a default value to the proxy
                # trait as would be expected of the public one were there
                # no go between applied to it.
                default = getattr(type(owner), self.trait_name).default()
                setattr(owner, self.proxy, default)
            return func(owner)
        self.func = wrapper
        return self

    def instance_init(self, obj):
        # reset the value after a default is generated
        # and after a new value is assigned.
        obj.observe(self._reset, self.trait_name, "change")
        obj.observe(self._reset, self.trait_name, "default")

    def _reset(self, data):
        try:
            # delete the value which is currently present
            # doing so will cause the default generator to
            # be called again.
            del data.owner._trait_values[data.name]
        except:
            pass
        else:
            if hasattr(data, "new"):
                # the data is a change notification and
                # we assign the new value to a proxy trait
                # which was specified in the decorator.
                setattr(data.owner, self.proxy, data.new)
            # otherwise it was a default notification
            # and we don't need to worry about it

In [18]:
class A(HasTraits):
    
    standard_transform = Int()
    other_special_transform = Int()
    
    special_transform = Int(read_only=True)
    
    @go_between("special_transform")
    def get_a(self):
        return self.standard_transform + self.other_special_transform
    
